### Entropy
* Shannon Entropy $H$, defined as: $ H = -\sum_{i=1}^n P(x_i)log_2 P(x_i) $  
    * decreases as number of possible outcomes decreases

In [ ]:
#need to do:
conda install -c anaconda graphviz

### Decision Trees
* Downsides:
    * ramdomness in generation, which can lead to variance in estimates. Also not built same way each time.
    * **they are incredibly prone to overfitting**, especially if  deep or complex. 
    * Because they work from info gain, they are biased towards the dominant class, so **balanced data is needed**.
* ID3 algorithm
    * requirements:
        * binary outcome
        * categorical features
    * splits on the most valuable attribute (causes largest drop of H) in each feature
* Random forests
    * create many trees and use them to vote
    * uses bagging to get better diversity
    * uses random subspace - only uses some of the features on each tree
        * generally for a dataset with x features $\sqrt{x}$ features are used for classifiers and $x/3$ for regression.
    * only works on test values that are within the training values
    * black box algo

## Support Vector Machines (SVMs)
* __S__upport __V__ector __C__lassifiers
* builds hyperplane
    * A hyperplane in n-dimensional space is an (n-1)-dimensional space.
* soft margin in when can't classify observations on one side or the other
    * SVC deals with this by cost function:
        * maximize margin size vs 
        * minimize cumulative distance of points on wrong side from boundary
* kernel trick 
    * unrelated to computer kernels
    * kernels map data to space using weights. eg kernel smoothing.
    * in SVM a kernel is a function that implicitly computes the dot product between two vectors in a higher-dimensional space without actually transforming the vectors into that space.
    * additional reading:
        * post [post by Eric Kim](http://www.eric-kim.net/eric-kim-net/posts/1/kernel_trick.html), 
        * then this [paper by Martin Hoffman](http://www.cogsys.wiai.uni-bamberg.de/teaching/ss06/hs_svm/slides/SVM_Seminarbericht_Hofmann.pdf). 
        * This [hour lecture from Patrick Winston](https://www.youtube.com/watch?v=_PwhiWxHK8o) is a good short-breathed derivation of everything you've covered in SVM so far. 
        * If you don't have a good intuition of why dot products are important, check out this [video by Sal Khan](https://www.khanacademy.org/math/linear-algebra/vectors-and-spaces/dot-cross-products/v/vector-dot-product-and-vector-length) from Khan Academy's linear algebra sequence.
    * kernel choice
        * default kernel - radial basis function, which uses a Gaussian decay according to the distance from the original point. 
        * other kernels: choose if [geometry justifies](https://stats.stackexchange.com/a/18032)
        * can use cross validation in training set, but if tuning other hyperparameters, can be time consuming
* SVM extensions
    * simplest: hold-on-out on each classification (class vs all others), get confidence estimate for each, and classification with highest estimate (based on distance to boundary and classifier accuracy) wins
    * pairwise boundaries; classification based on highest 1-vs-1 wins count
* __S__upport __V__ector __R__egression
    * works in reverse: penalty based on distance from far points
    * parameters
        * C: box constraint and sets the penalty for being outside of our margin. 
        * Epsilon: sets the size of our margin. 
    * advantage: can set sensitivity before model creation, not just after


### Meshgrid example
Used for SVC or KNN classifier  
Example for 22.2
```
# Visualize our model
y_min, y_max = X.test.min() - 1, X.test.max() + 3
x_min, x_max = X.project.min() - 1, X.project.max() + 3
xx, yy = np.meshgrid(np.arange(x_min, x_max, .1),
                     np.arange(y_min, y_max, .1))

Z = (svm.predict(np.c_[xx.ravel(), yy.ravel()])=='pass')

Z = Z.reshape(xx.shape)
plt.contourf(xx, yy, Z, cmap=plt.cm.coolwarm, alpha=0.8)
plt.scatter(test_data.project[0:10], test_data.test[0:10], marker='x')
plt.scatter(test_data.project[10:20], test_data.test[10:20], marker='o')
plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)
plt.xlabel('Project Grade')
plt.ylabel('Test Grade')
plt.title('Passing Grades SVM Example')
plt.show()
```

## Boosting
* very popular for use
* boosting models vary by:
    * Type of simple model; almost any work.
    * Index of error. You can use residuals from regression, classification errors, or any cost function.
    * How the next iteration targets the error. You can weight inaccurately-predicted cases high and accurately-predicted cases low, you can directly model residuals, or you can model only the subset of the data that was inaccurately predicted.
    * Stopping rule. You can stop once you've run a certain number of models, once the amount of variance explained by the most recent iteration of the model is lower than some threshold, or once the change in weights between the two most recent model iterations is lower than some threshold.
* Gradient boosting can work on any combination of loss function and model type, as long as we can calculate the derivatives of the loss function with respect to the model parameters. 
* for regression trees:
    * minimize $\frac1{n}\sum_{i=1}^n(y_i-(\alpha + \beta x_i))^2$
    * takes residual of previous weak learner tree and tries to predict that. loop.
    * often better than random forests, and less prone to overfitting that individual trees.
        * can also subsample, so each iteration only gets part of the data
        * can use shrinkage (a la Ridge regr) to lower impact of subsequent submodels
            * learning rate 0 means only final combined model matters,
            * learning rate 1 - all weighted equally

### Ensemble models

   * made up of other models
   * types:
       * bagging aka "boostrap aggregating" -drawing with replacement
       * boosting - serial models daisy-chained together
       * multiple models as first step, then output of these used as input for final model. Combination of other two methods
    * 
    

### Cross validation, leakage, etc.
* training - 80% of subset. don't use training error. 
* validation - used to select hyperparameters or model type.
* testing - held out data used to see how good model is. 
* kfold
    * Cross validation samples without replacement
    * Boostrap - samples ___with___ replacement
    * used to get multiple estimates of prediction error which can be averaged. 
* sequence:
    * determine hyperparameters with grid/random search 
        * picks set of hyperparameters for training subset, finds parameters, and validates on remaining. 
        * returns best hyperparameters.
    * retrain on entire training dataset to get parametrs ?
    * get error estimate on test data 
    * put into production
* nested cross-validation:
    * outer loop creates inner loops
    * inner loops 
        * multiple times through the data set, setting aside different 20% for validation. 
        * Each time:
            * Using the 80%, run grid/random search to repeatedly find parameter for each set of hypers, and choose best hypers.
        * then you get average 
    * outer loop through multiple instances of:
        use inner loops to find hypers. train on whole 'inner train&test' set and test again held out data.
    * hopefully the best hypers found are close or identical. if not, it is an instability problem, although if multiple hypers are yeilding similar results, they may simply be unimportant.
    * probably unneeded on classifiers with few parameters, or for random forest, SVM with Gaussian kernel, and gradient boosting machines